In [ ]:
import os
import requests
from lxml import html
from bs4 import BeautifulSoup
import tabula
from tabula import wrapper
from datetime import datetime
import pandas as pd
import wget

##데이터 crawling Url 생성 함수
def makeURL(myUrl, myKey, myParameter):
    # myUrl = "http://192.168.1.120/index.php?"
    url = myUrl + myKey + "&" + myParameter

    url = url.rstrip('&')
    return url

##xml형석 검사 함수
def xmlProcess(url):
    response = requests.get(url)
    # Check if page is up
    if response.status_code == 200:
        # Convert webpage to %Data
        Data = BeautifulSoup(response.text, 'lxml-xml')
        result = []
        rows = 0
        columnName = []
        # search Item all item tag
        iterData = Data.find_all('item')
        for item in iterData:
            item_list = []
            # Fill the value in one row
            for tag in item.find_all():
                try:
                    tagname = tag.name
                    if rows == 0:
                        columnName.append(tagname)
                    item_list.append(item.find(tagname).text)
                except Exception as e:
                    print("This row will be ignored. ", item_list)
            rows = rows + 1
            result.append(item_list)
    finalResult = pd.DataFrame(result)
    finalResult.columns = columnName
    print(finalResult)
    return finalResult

##Json형식 검사함수
def jsonProcess(url):

    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    # JSON 데이터 획득
    json = response.json()
    # PandasDataframe변환
    df = json_normalize(json)
    return df

##csv형식 검사함수
def csvProcess(url):
    # 정상 여부 확인 (200 정상)
    response = requests.get(url)
    df = pd.read_csv(url, encoding="ms949")
    return df

##pdf파일을 받아 저장까지 완료하는 함수
def pdfProcess(inputFolder,url):
    resp = requests.get("http://fsc.go.kr/info/trd_list.jsp?menu=7230000&bbsid=BBS0069")
    resp.encoding='utf-8'
    html = resp.text
    bs = BeautifulSoup(html, 'html.parser')

    #프로그램 실행시마다 최신데이터를 데이터를 가져오도록 하는 코드 
    #데이터 추출하기
    originalData  = bs.select("#contents > div.board > table > tbody > tr > td > a")

    #pdf문서
    convertedData = str(originalData[2])

    dailyUpdated = convertedData.split('"')[1].split('amp;')[1]
#     url = "http://fsc.go.kr/downManager?bbsid=BBS0069&"
    url = url + "&"+ dailyUpdated
    df2 = wrapper.read_pdf(url,
              multiple_tables=True,
              pages="all",pandas_options={"header":0})

    #csv 파일로 저장하기 
    #파일이름에 반영할 현재 날짜 구하기 
    currentYear = datetime.today().year
    currentMonth = datetime.today().month
    currentDate = datetime.today().day

    currentYear

    currentYear = str(currentYear)[-2:]
    if (currentMonth < 10):
        currentMonth = "0" + str(currentMonth)
    else:
        currentMonth = str(currentMonth)

    if (currentDate < 10):
        currentDate = "0" + str(currentDate)
    else:
        currentDate = str(currentDate)

    today = currentYear + currentMonth + currentDate

    #################################저장 수정필요
    for i in range(len(df2)):
        fileName = inputFolder + "_" + today + "_" + str(i)
        fullOutPath = outPath+inputFolder+"/"+inputFolder+inputFile+str(i)+".csv"
    #         df2[i].to_sql(fileName, engine, if_exists='replace', index=False)
        df2[i].to_csv(fullOutPath, encoding = "ms949", index = False)

##데이터 폴더 생성 함수
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

##구글 스프레드 시트 활용
dataList = pd.read_csv("") ## googleSheet link연결 서비스키 보안
print("### The total number of target data is " + str(len(dataList)))

dataList = dataList[dataList['사이트'].notnull() ]
print(dataList[["사이트"]])
print("### The total number of filtered data is " + str(len(dataList)))

dataList = dataList[   dataList['폴더명'] == "342_금융시장동향" ]
dataList

outPath = "./outbound/"
folderList = dataList["폴더명"].tolist()

for i in folderList:
    createFolder(outPath+i)

dataList = dataList.fillna("")
dataList = dataList.reset_index(drop=True)

for dataCount in range(0,len(dataList)):
    inputUrl = dataList.loc[dataCount, "사이트"]
    inputKey = dataList.loc[dataCount, "서비스키"]
    inputParameter = dataList.loc[dataCount, "파라미터"]
    inputFolder = dataList.loc[dataCount, "폴더명"]
    inputFile = dataList.loc[dataCount, "서비스명"]
    inputDataType = dataList.loc[dataCount, "데이터타입"]
    inputRefUrl = dataList.loc[dataCount, "참고문서"]
    inputRefType = dataList.loc[dataCount, "참고문서타입"]
    print(inputUrl)

    url = makeURL(inputUrl,inputKey,inputParameter)

    print("fullUrl is " + url)

    newDF = pd.DataFrame()
    try:
        if (inputDataType == "xml"):
            try:
                newDF = cf.xmlProcess(url)
            except Exception as e:
                pass
        elif(inputDataType == "json"):
            newDF = jsonProcess(url)
        elif(inputDataType == "csv"):
            newDF = csvProcess(url)
        elif(inputDataType == "pdf"):
            newDF = pdfProcess(inputFolder, url)
        print("저장 되었습니다")
    except Exception as a:
        print(a)
        pass